# Feature selection

In this experiment, we will be comparing a stepwise forward selection method to alternative techniques such as L1 regularization and PLS regression.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from utils import stepwise_forward_regression, pls_cross_validation, simulate_data
import numpy as np
import statsmodels.api as sm
import json
from sklearn.preprocessing import StandardScaler

np.set_printoptions(suppress=True)

First, we will simulate some data with 100 variables and 10K observations. Only the first 10 variables will be used to generate the output. The output will be sampled from a binomial distribution, and the beta coefficients will be drawn from a uniform distribution.

In [ ]:
# Simulate a process with 100 variables and 10K observations. Only 10 variables are correlated with the dependent variable
np.random.seed(1234)
n = 500
p = 100
intercept = -2

X, y, beta, var_names = simulate_data(n, p, intercept, 10)

X_standardized = StandardScaler().fit_transform(X)
# Generate the dependent using an inverse logit function and round to the closest integer
y_prob = 1 / (1 + np.exp(-X_standardized@beta-intercept))

real_auc = roc_auc_score(y, y_prob)
print("Real coefficients", beta)
print("Real AUC: ", real_auc)

The first model to test is a Logistic Regression with no feature selection. Although the significant coefficients have similar values to the simulated ones, we can see that some of them are a little bit off, and that some of the coefficients that are supposed to be 0 are sometimes slightly different than 0.

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

ss = StandardScaler()
X_train = pd.DataFrame(ss.fit_transform(X_train), columns=var_names)
X_test = pd.DataFrame(ss.transform(X_test), columns=var_names)
# Scores of the initial model using all the variables
base_lr = LogisticRegression()
base_lr.fit(X_train, y_train)
print("AUC score", roc_auc_score(y_test, base_lr.predict_proba(X_test)[:, 1]))
print("Coefficients", base_lr.coef_[0])
print("Intercept", base_lr.intercept_[0])

In [ ]:
# Perform stepwise forward regression
selected_features = sorted(stepwise_forward_regression(X_train, y_train))
print("Selected features", selected_features)

In [ ]:
# Performance of the model given the selected features
stepwise_lr = LogisticRegression()
stepwise_lr.fit(X_train[selected_features], y_train)
print("AUC score", roc_auc_score(y_test, stepwise_lr.predict_proba(X_test[selected_features])[:, 1]))

print("Coefficients", stepwise_lr.coef_[0])
print("Intercept", stepwise_lr.intercept_[0])

In [ ]:
# Fit the model with L1 regularization
l1_lr = LogisticRegression(penalty='elasticnet', l1_ratio=.1,solver='saga')
l1_lr.fit(X_train, y_train)

# Print the coefficients
print("Coefficients", l1_lr.coef_[0])
print("Intercept", l1_lr.intercept_[0])

In [ ]:
comps, vips = pls_cross_validation(X_train, y_train, range(1, 10))
# %%
pls_lr = LogisticRegression()
pls_lr.fit(X_train.loc[:, vips > 1], y_train)
print(roc_auc_score(y_test, pls_lr.predict_proba(X_test.loc[:, vips > 1])[:, 1]))

print("Coefficients", pls_lr.coef_[0])
print("Intercept", pls_lr.intercept_[0])

In [ ]:
results = {
    "simulated":{
        "auc": real_auc,
        "n_features": p,
        "intercept": intercept,
        "coefficients": beta
    },
    "base_lr": {
        "coefficients": base_lr.coef_[0],
        "intercept": base_lr.intercept_[0],
        "auc": roc_auc_score(y_test, base_lr.predict_proba(X_test)[:, 1]),
        "selected_features": X.columns.tolist(),
        "n_features": int(p)  # Convert to Python int
    },
    "stepwise_lr": {
        "coefficients": stepwise_lr.coef_[0],
        "intercept": stepwise_lr.intercept_[0],
        "auc": roc_auc_score(y_test, stepwise_lr.predict_proba(X_test[selected_features])[:, 1]),
        "selected_features": selected_features,
        "n_features": int(len(selected_features))  # Convert to Python int
    },
    "pls_lr": {
        "coefficients": pls_lr.coef_[0],
        "intercept": pls_lr.intercept_[0],
        "auc": roc_auc_score(y_test, pls_lr.predict_proba(X_test.loc[:, vips > 1])[:, 1]),
        "selected_features": X.columns[vips > 1].tolist(),
        "n_features": int(sum(vips > 1))  # Convert to Python int
    }
}

In [ ]:
# Create a dataframe based on the results, using the keys as rows
df = pd.DataFrame(results).T

In [ ]:
df

In [ ]:
df['meaningful_coefficients'] = df['coefficients'].apply(lambda x: x[:10])

In [ ]:
meaningful_coeffs = np.stack(df['meaningful_coefficients'].values)

plt.plot(meaningful_coeffs.T)
plt.xlabel('Feature')
plt.ylabel('Coefficient')
plt.xticks(range(10), ['V{:02d}'.format(i) for i in range(1, 11)])
plt.legend(df.index)

In [ ]:
model_coeffs = meaningful_coeffs[0]
print("Norm of the difference between the coefficients of the first model and the current model")
for index, vector in enumerate(meaningful_coeffs):
    print(df.index[index], np.linalg.norm(model_coeffs-vector))